In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from environment.settings import config
from sklearn.cluster import KMeans
from sqlalchemy import select, text, and_, or_
from sqlalchemy.sql import Select
from utils import connections
from utils import database

database_dir = config['DATABASE_DIR']

We need to group the artworks by:
- Artist
- Movement
- Century
- Country

Get the group data from the database

In [ ]:
query = select(database.Artist)

with connections.session_db() as con:
    df = pd.read_sql_query(sql=query, con=con.connection())